In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import shutil

# 경고 뜨지 않게..
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 랜덤 모듈 
import random

# 학습 모델 저장 및 복원
import pickle

# 딥러닝 라이브러리
import tensorflow as tf
# 신경망 모델을 관리하는 객체
from tensorflow.keras.models import Sequential
# 선형 회귀 레이어
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 객체
from tensorflow.keras.layers import Activation
# CNN : 커널을 통해 합성곱을 구하는 것. 이미지의 특징이 두드러 지게 한다.
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv1D
# MaxPooling : 커널내에서 가장 큰 값을 추출하는 방식으로 불필요한 부분을 제거한다.
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import MaxPool1D
# Flatten : 다차원의 이미지 데이터를 선형 회귀 은닉층으로 전달하기 전에
# 1차원으로 변환하는 것
from tensorflow.keras.layers import Flatten
# Dropout : 이미지나 영상, 음파 등의 데이터는 오랫동안 학습을 진행 시켜야 한다.
# 하지만 너무 빨리 과적합이 발생되면 조기 중단 때문에 학습 횟수가 줄어들 수 있다.
# 이에 은닉의 노드를 일부 사용하지 않으므로써 과적합이 빨리 오는 것을 예방하고
# 오랫동한 학습이 진행될 수 있도록 한다.
from tensorflow.keras.layers import Dropout
# Embedding : 단어의 수를 조정한다.
from tensorflow.keras.layers import Embedding

# LSTM
from tensorflow.keras.layers import LSTM



# 원핫 인코딩을 수행하는 함수
from tensorflow.keras.utils import to_categorical

# 저장된 학습모델을 복원한다.
from tensorflow.keras.models import load_model

# 모델을 자동 저장한다.
from tensorflow.keras.callbacks import ModelCheckpoint
# 성능이 더이상 좋아지지 않을 경우 중단 시킨다.
from tensorflow.keras.callbacks import EarlyStopping

# 문장을 잘라준다.
from tensorflow.keras.preprocessing.text import Tokenizer
# 모든 문장 데이터의 단어 데이터의 수를 동일한 수로 맞춰준다.
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 문자열을 가지고 단어 사전을 만들어준다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence



# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# 문자열 -> 숫자
from sklearn.preprocessing import LabelEncoder

# 전체데이터를 학습용과 검증으로 나눈다.
from sklearn.model_selection import train_test_split

# 랜덤시드 설정
# 데이터를 랜덤하게 섞거나 가중치를 랜덤하게 설정하는 등..
# 작업에서 랜덤을 적용하는 경우가 더러 있다.
# 이에, 시드를 고정시킨다.
random_seed = 1

np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

# 현재 프로젝트에서 GPU 메모리 사용을 필요한 만큼만 쓸 수 있도록 한다.
# 컴퓨터에 있는 GPU 정보들을 가져온다.
gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면...
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

### 데이터를 읽어온다.

In [2]:
df = pd.read_csv('./data/review_data.csv')
df

,score,review,y
0,5,친절하시고 깔끔하고 좋았습니다,1
1,5,조용하고 고기도 굿,1
2,4,"갈비탕과 냉면, 육회비빔밥이 맛있습니다.",1
3,4,대체적으로 만족하나\n와인의 구성이 살짝 아쉬움,1
4,5,고기도 맛있고 서비스는 더 최고입니다~,1
...,...,...,...
540,3,추웟어요 고기 외에는 별로에요..,0
541,1,고기질과 육전은 좋다.다만 한우손님 돼지고기 손님을 차별한다(돼지손님은 주차불가.네...,0
542,5,직접 구워주시고 진짜맛있음. 반찬도 맛있음. 직원분이 친절하게 잘해주시네요,1
543,4,친절하게 서빙해주시고 음식도 챙겨주셨어요 ㅎ,1


### 한글 외의 모든 글자를 제거한다.

In [5]:
import re
def text_clearning(text):
    # 한글 정규식(띄어쓰기, ㄱ~ㅣ, 가~힣)
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    # 정규식에 위배되는 것들은 길이가 0인 문자열로 변환한다.
    result = hangul.sub('', text)
    
    return result

In [6]:
# 리뷰 내용을 저장한다.
df['ko_review'] = df['review'].apply(lambda x : text_clearning(x))
# review 컬럼은 제거한다.
df.drop('review', axis=1, inplace=True)
df

,score,y,ko_review
0,5,1,친절하시고 깔끔하고 좋았습니다
1,5,1,조용하고 고기도 굿
2,4,1,갈비탕과 냉면 육회비빔밥이 맛있습니다
3,4,1,대체적으로 만족하나와인의 구성이 살짝 아쉬움
4,5,1,고기도 맛있고 서비스는 더 최고입니다
...,...,...,...
540,3,0,추웟어요 고기 외에는 별로에요
541,1,0,고기질과 육전은 좋다다만 한우손님 돼지고기 손님을 차별한다돼지손님은 주차불가네이버예...
542,5,1,직접 구워주시고 진짜맛있음 반찬도 맛있음 직원분이 친절하게 잘해주시네요
543,4,1,친절하게 서빙해주시고 음식도 챙겨주셨어요 ㅎ


### 형태소 분석

In [7]:
from konlpy.tag import Okt

In [11]:
# 형태소를 분리해서 데이터를 만드는 함수
tagger = Okt()
def get_pos(x):
    # 매개변수로 들어온 문장의 품사를 구한다.
    pos = tagger.pos(x)
    
    # 단어와 품사를 합쳐서 하나의 단어로 만든다.
    result = []
    
    # 형태소의 수 만큼 반복한다.
    for a1 in pos:
        result.append(f'{a1[0]}/{a1[1]}')
        
    return result

In [10]:
tagger.morphs('동해물과 백두산이 마르고 닳도록')

['동', '해물', '과', '백두산', '이', '마르고', '닳도록']

In [9]:
tagger.pos('동해물과 백두산이 마르고 닳도록')

[('동', 'Modifier'),
 ('해물', 'Noun'),
 ('과', 'Josa'),
 ('백두산', 'Noun'),
 ('이', 'Josa'),
 ('마르고', 'Noun'),
 ('닳도록', 'Verb')]

In [12]:
get_pos('동해물과 백두산이 마르고 닮도록')

['동/Modifier',
 '해물/Noun',
 '과/Josa',
 '백두산/Noun',
 '이/Josa',
 '마르고/Noun',
 '닮도록/Verb']

### 학습 데이터를 생성한다.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [17]:
# 단어 사전을 만든다(각 단어별로 숫자를 부여한다.
index_vectorizer = CountVectorizer(tokenizer=lambda x : get_pos(x))
X = index_vectorizer.fit_transform(df['ko_review'].tolist())
X

<545x3030 sparse matrix of type '<class 'numpy.int64'>'
	with 9692 stored elements in Compressed Sparse Row format>

In [18]:
index_vectorizer.vocabulary_

{'친절하시고/Adjective': 2647,
 '깔끔하고/Adjective': 428,
 '좋았습니다/Adjective': 2403,
 '조용하고/Adjective': 2356,
 '고기/Noun': 233,
 '도/Josa': 721,
 '굿/Noun': 330,
 '갈비탕/Noun': 120,
 '과/Josa': 260,
 '냉면/Noun': 528,
 '육회/Noun': 2065,
 '비빔밥/Noun': 1419,
 '이/Josa': 2082,
 '맛있습니다/Adjective': 1013,
 '대/Modifier': 671,
 '체적/Noun': 2604,
 '으로/Josa': 2067,
 '만족하나/Adjective': 956,
 '와인/Noun': 1996,
 '의/Josa': 2077,
 '구성/Noun': 293,
 '살짝/Noun': 1476,
 '아쉬움/Noun': 1705,
 '맛있고/Adjective': 1001,
 '서비스/Noun': 1508,
 '는/Josa': 589,
 '더/Noun': 701,
 '최고/Noun': 2613,
 '입니다/Adjective': 2182,
 '가/Josa': 24,
 '입/Noun': 2177,
 '에서/Josa': 1897,
 '녹아요/Verb': 553,
 '였습니다/Verb': 1935,
 '살살/Noun': 1473,
 '녹는/Verb': 552,
 '최상급/Noun': 2615,
 '소고기/Noun': 1540,
 '를/Josa': 901,
 '맛/Noun': 995,
 '보고왔습니다/Verb': 1316,
 '구워주고/Verb': 302,
 '가성/Noun': 73,
 '비/Noun': 1408,
 '짱/Noun': 2528,
 '콜키/Noun': 2688,
 '지/Josa': 2477,
 '프리/Noun': 2788,
 '라서/Josa': 885,
 '을/Josa': 2070,
 '가지/Noun': 89,
 '고/Josa': 227,
 '가면/Noun': 61,
 '좋은/Adjective

In [19]:
# 단어사전을 기반으로 입력 데이터를 생성한다.
t1 = TfidfTransformer()
X = t1.fit_transform(X)
print(X[0])

  (0, 2647)	0.5548708693511647
  (0, 2403)	0.48955631270748484
  (0, 428)	0.6726462183300624


In [20]:
# 결과 데이터
y = df['y']
y

0      1
1      1
2      1
3      1
4      1
      ..
540    0
541    0
542    1
543    1
544    1
Name: y, Length: 545, dtype: int64

### 학습한다

In [21]:
# 기본
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 경고가 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차검증
# 지표를 하나만 설정할 경우
from sklearn.model_selection import cross_val_score
# 지표를 하나이상 설정할 경우
from sklearn.model_selection import cross_validate

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가함수 - 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 평가함수 - 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 머신러닝 알고리즘 - 분류용
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀용
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 시간 측정을 위한모듈
import datetime
# 저장을 위한 모듈
import pickle

### 기본 모델 사용하기

In [27]:
# KFold 생성
kfold = KFold(n_splits=10, shuffle=True, random_state=1)

In [22]:
# 사용할 모델들
model1 = KNeighborsClassifier()
model2 = LogisticRegression(random_state=1)
model3 = SVC(random_state=1, probability=True)
model4 = DecisionTreeClassifier(random_state=1)
model5 = RandomForestClassifier(random_state=1)
model6 = AdaBoostClassifier(random_state=1)
model7 = GradientBoostingClassifier(random_state=1)
model8 = LGBMClassifier(random_state=1)
model9 = XGBClassifier(random_state=1, silent=True, verbosity=0)

model_list1= [
    ('m1', KNeighborsClassifier()),
    ('m2', LogisticRegression(random_state=1)),
    ('m3', SVC(random_state=1, probability=True)),
    ('m4', DecisionTreeClassifier(random_state=1)),
    ('m5', RandomForestClassifier(random_state=1)),
    ('m6', AdaBoostClassifier(random_state=1)),
    ('m7', GradientBoostingClassifier(random_state=1)),
    ('m8', LGBMClassifier(random_state=1)),
    ('m9', XGBClassifier(random_state=1, silent=True, verbosity=0))
]
# 배깅 운영방식은 위에서 사용한 앙상블, 부스팅들이 사용하고 있다.
# 보팅 운영방식도 테스트해보기 위해 사용한다.
# 매개변수 voting은 투표방식을 설정한다 (hard, soft)
model10 = VotingClassifier(estimators=model_list1, voting='hard')

model_list2 = [
    ('m10', KNeighborsClassifier()),
    ('m20', LogisticRegression(random_state=1)),
    ('m30', SVC(random_state=1, probability=True)),
    ('m40', DecisionTreeClassifier(random_state=1)),
    ('m50', RandomForestClassifier(random_state=1)),
    ('m60', AdaBoostClassifier(random_state=1)),
    ('m70', GradientBoostingClassifier(random_state=1)),
    ('m80', LGBMClassifier(random_state=1)),
    ('m90', XGBClassifier(random_state=1, silent=True, verbosity=0))
]

model11 = VotingClassifier(estimators=model_list2, voting='soft')

In [23]:
# 성능 값을 담을 리스트
score_list = []
# 학습 모델의 이름을 담을 리스트
model_name_list = []
# 최적으로 튜닝된 학습 모델을 담을 리스트
best_model_list = []

In [24]:
# 교차 검증을 수행할 학습 모델 리스트
check_list = [
    model1, model2, model3, model4, model5, model6, model7,
    model8, model9, model10, model11
]
# 모델의 이름
check_name_list = [
    'KNN (Basic)', 'LogisticRegression (Basic)', 'SVM (Basic)',
    'DecisionTree (Basic)', 'RandomForest (Basic)', 'AdaBoost (Basic)',
    'GradientBoost (Basic)', 'LGBM (Basic)', 'XGBoost (Basic)',
    'Voting (Hard) (Basic)', 'Voting (Soft) (Basic)'
]

In [28]:
# 교차 검증
for idx, ml in enumerate(check_list) :
    score = cross_val_score(ml, X, y, scoring='f1', cv=kfold)
    
    # 현재 알고리즘의 이름과 평가 점수의 평균을 출력한다.
    print(f'{check_name_list[idx]} : {score.mean()}')
    
    # 현재 알고리즘의 이름과 평가 점수의 평균을 리스트에 담는다.
    score_list.append(score.mean())
    model_name_list.append(check_name_list[idx])
    best_model_list.append(ml)

KNN (Basic) : 0.9525917258566562
LogisticRegression (Basic) : 0.9479277809113741
SVM (Basic) : 0.9479277809113741
DecisionTree (Basic) : 0.9362976273864975
RandomForest (Basic) : 0.9479277809113741
AdaBoost (Basic) : 0.948876354535771
GradientBoost (Basic) : 0.9481445809833267
LGBM (Basic) : 0.9494364691060998
XGBoost (Basic) : 0.9459729466994645
Voting (Hard) (Basic) : 0.9488513204302744
Voting (Soft) (Basic) : 0.9525702466113903


In [32]:
# 전체 데이터를 학습한다.
model2.fit(X, y)

LogisticRegression(random_state=1)

In [35]:
# 학습때 사용한 각 컬럼의 중요도
# 1과 가까울수록 컬럼이 중요한 것이고
# 0과 가까울수록 컬럼이 중요한 것이 아니다.
model2.coef_[0]

array([ 0.05961542,  0.12712938,  0.14690943, ...,  0.01883734,
       -0.26061158,  0.027145  ])

In [36]:
# 예측할 문장들
test_data = [
    '음식이 맛있어요',
    '음식이 맛없어요',
    '식당이 너무 좋아요',
    '식당이 너무 나빠요'
]

In [38]:
X = index_vectorizer.transform(test_data)
X = t1.transform(X)

In [39]:
# 예측한다.
y_pred = model2.predict(X)
y_pred

array([1, 1, 1, 1], dtype=int64)